In [1]:
import yfinance as yf
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import loadStocks as ls
import featEng as fe

In [3]:
TICKER_NAMES = ['fb','msft']
STOCKS = ls.load_tickers_df(TICKER_NAMES, display="append")

df_grouping = []
for n in fe.DataframHandler(STOCKS, fe.getHistoricReturns).fe_objects:
    df_grouping.append(n.data.tail(365))

In [6]:
df_grouping[1]['365_price_ago']

3576    51.13
3577    51.14
3578    51.40
3579    52.15
3580    52.09
        ...  
3936    72.19
3937    72.04
3938    72.31
3939    71.83
3940    71.13
Name: 365_price_ago, Length: 365, dtype: float64

In [10]:
log_ret = pd.Series(df_grouping[1]['365_price_ago'])
log_ret.shape

(365,)

In [11]:
np.random.seed(42)
num_ports = 6000
all_weights = np.zeros((num_ports, len(df_grouping[1].columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for x in range(num_ports):
    # Weights
    weights = np.array(np.random.random(8))
    weights = weights/np.sum(weights)
    
    # Save weights
    all_weights[x,:] = weights
    
    # Expected return
    ret_arr[x] = np.sum( (log_ret.mean() * weights * 252))
    
    # Expected volatility
    vol_arr[x] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov()*252, weights)))
    
    # Sharpe Ratio
    sharpe_arr[x] = ret_arr[x]/vol_arr[x]

TypeError: cov() missing 1 required positional argument: 'other'

In [ ]:
print("Max Sharpe ratio in the array: {}".format(sharpe_arr.max()))
print("It's location in the array: {}".format(sharpe_arr.argmax()))

In [ ]:
print(all_weights[5889,:])
max_sr_ret = ret_arr[sharpe_arr.argmax()]
max_sr_vol = vol_arr[sharpe_arr.argmax()]

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(vol_arr, ret_arr, c=sharpe_arr, cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
plt.scatter(max_sr_vol, max_sr_ret,c='red', s=50) # red dot
plt.show()